In [ ]:
//Imports
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.ml.feature.RegexTokenizer
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.CountVectorizer
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

  

>     import org.apache.spark.ml.feature.StopWordsRemover
>     import org.apache.spark.ml.feature.RegexTokenizer
>     import org.apache.spark.ml.classification.LogisticRegression
>     import org.apache.spark.ml.feature.CountVectorizer
>     import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

In [ ]:
/scalable-data-science/000_0-sds-3-x-projects/student-project-01_group-TheTwoCultures/01_load_data

  

>     import org.apache.spark.sql.functions.{col, concat_ws, udf, flatten, explode, collect_list, collect_set, lit}
>     import org.apache.spark.sql.types.{ArrayType, StructType, StructField, StringType, IntegerType}
>     import com.databricks.spark.xml._
>     import org.apache.spark.sql.functions._
>     read_xml: (file_name: String)org.apache.spark.sql.DataFrame
>     get_dataset: (file_name: String)org.apache.spark.sql.DataFrame

  

>     save_df: (df: org.apache.spark.sql.DataFrame, filePath: String)Unit
>     load_df: (filePath: String)org.apache.spark.sql.DataFrame
>     no_forums: (df: org.apache.spark.sql.DataFrame)Long

  

>     dbfs:/datasets/student-project-01/flashback/familjeliv-allmanna-ekonomi_df
>     familjeliv-allmanna-ekonomi_df already exists!
>     dbfs:/datasets/student-project-01/flashback/familjeliv-sexsamlevnad_df
>     familjeliv-sexsamlevnad_df already exists!
>     dbfs:/datasets/student-project-01/flashback/flashback-ekonomi_df
>     flashback-ekonomi_df already exists!
>     dbfs:/datasets/student-project-01/flashback/flashback-sex_df
>     flashback-sex_df already exists!
>     fl_root: String = dbfs:/datasets/student-project-01/familjeliv/
>     fb_root: String = dbfs:/datasets/student-project-01/flashback/
>     fl_data: Array[String] = Array(familjeliv-allmanna-ekonomi, familjeliv-sexsamlevnad)
>     fb_data: Array[String] = Array(flashback-ekonomi, flashback-sex)

In [ ]:
//Load dataframes
val file_path_familjeliv = "dbfs:/datasets/student-project-01/familjeliv/familjeliv-sexsamlevnad_df"
val file_path_flashback = "dbfs:/datasets/student-project-01/flashback/flashback-sex_df"
val df_familjeliv = load_df(file_path_familjeliv)
val df_flashback = load_df(file_path_flashback)

  

>     file_path_familjeliv: String = dbfs:/datasets/student-project-01/familjeliv/familjeliv-sexsamlevnad_df
>     file_path_flashback: String = dbfs:/datasets/student-project-01/flashback/flashback-sex_df
>     df_familjeliv: org.apache.spark.sql.DataFrame = [thread_id: string, thread_title: string ... 5 more fields]
>     df_flashback: org.apache.spark.sql.DataFrame = [thread_id: string, thread_title: string ... 5 more fields]

In [ ]:
//Extract the text
val df_text_flashback = df_flashback.select("w")
val df_text_familjeliv = df_familjeliv.select("w")


  

>     df_text_flashback: org.apache.spark.sql.DataFrame = [w: string]
>     df_text_familjeliv: org.apache.spark.sql.DataFrame = [w: string]

In [ ]:
//Add classification columns
val df_text_flashback_c = df_text_flashback.withColumn("c", lit(0))
val df_text_familjeliv_c = df_text_familjeliv.orderBy(rand()).limit(df_text_flashback_c.count().toInt).withColumn("c", lit(1))
val df_text_full = df_text_flashback_c.union(df_text_familjeliv_c)
println(df_text_flashback_c.count())
println(df_text_familjeliv_c.count())
println(df_text_full.count())

  

>     56621
>     56621
>     113242
>     df_text_flashback_c: org.apache.spark.sql.DataFrame = [w: string, c: int]
>     df_text_familjeliv_c: org.apache.spark.sql.DataFrame = [w: string, c: int]
>     df_text_full: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [w: string, c: int]

In [ ]:
val tokenizer = new RegexTokenizer()
  .setPattern("(?U),") // break by whitespace
  .setMinTokenLength(5) // Filter away tokens with length < 5
  .setInputCol("w") // name of the input column
  .setOutputCol("text") // name of the output column
val tokenized_df = tokenizer.transform(df_text_full).select("c", "text")
tokenized_df.show(3, false)

  

>     +---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
>     |c  |text                                                                                                                                                                                                                                                                                                                                                                        |
>     +---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
>     |0  |[flyttad, filmforumet, någon, denna, blrjan, talet]                                                                                                                                                                                                                                                                                                                         |
>     |0  |[buttman-, film.jag, nästan, hundra, alltså, ingen, annan.sen, också, mig.buttman, filmar.jag, filmen, filmnet, nångång, 94-95]                                                                                                                                                                                                                                             |
>     |0  |[finns, hittar, främst, hubben, tjena, verkar, flippad, någon, provar, precis, undrade, <hub-security>, personuppgifter, finns, adolfsberg, ligger, letar, efter, bilder, eller, filmer, tjejer, amatör, privat, kvalitet, kommer, närke, eller, omnejd, kommer, bokstaven, samma, skola, främst, hubben, precisera, tacksam, dc-hub, borde, detta, räcka:x.organiccrap.com]|
>     +---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
>     only showing top 3 rows
>
>     tokenizer: org.apache.spark.ml.feature.RegexTokenizer = RegexTokenizer: uid=regexTok_744371e04826, minTokenLength=5, gaps=true, pattern=(?U),, toLowercase=true
>     tokenized_df: org.apache.spark.sql.DataFrame = [c: int, text: array<string>]

In [ ]:
//Stopwordsremover (similar to lda notebook)
val stoppord = sc.textFile("dbfs:/datasets/student-project-01/stoppord.csv").collect()
val stopwordList = Array("bara","lite","finns","vill","samt","inga","även","finns","ganska","också","igen","just","that","with","http","jpg",  "kanske","tycker","gillar","bra","000","måste","tjej","tjejer","tjejen","tjejerna","kvinna","kvinnor","kille","killar","killen","män","rätt","män","com","and","html","många","aldrig","www","mpg","avi","wmv","riktigt","känner","väldigt","font","size","mms","2008","2009", "flashback", "familjeliv").union(stoppord).union(StopWordsRemover.loadDefaultStopWords("swedish"))

val remover = new StopWordsRemover()
  .setStopWords(stopwordList)
  .setInputCol("text")
  .setOutputCol("filtered")


  

>     stoppord: Array[String] = Array(word, all, alla, allas, allt, alltså, andra, andras, annan, annat, artonde, artonn, att, av, bakom, bara, bland, blev, bli, blir, blivit, båda, bådas, både, dag, dagar, dagarna, dagen, de, del, delen, dem, den, denna, deras, dess, dessa, det, detta, dig, din, dina, dit, ditt, dock, dom, du, där, därför, då, efter, eftersom, ej, elfte, eller, elva, en, er, era, ert, ett, fall, fanns, fast, fem, femte, femtio, femtionde, femton, femtonde, fick, fin, finnas, finns, fjorton, fjortonde, fjärde, fler, flera, flesta, fram, framför, från, fyra, fyrtio, fyrtionde, få, får, fått, för, före, förra, första, ge, genom, ger, gick, gjorde, gjort, gälla, gäller, gällt, gärna, gå, gång, går, gått, gör, göra, ha, hade, haft, han, hans, har, hela, heller, hellre, helst, helt, henne, hennes, heter, hit, hjälp, hon, honom, hundra, hundraen, hundraett, hur, här, i, ibland, icke, igen, in, inför, inga, ingen, inget, innan, inne, inom, inte, inuti, ja, jag, ju, jämfört, kan, kom, komma, kommer, kommit, kunde, kunna, kunnat, kvar, kör, legat, ligga, ligger, lägga, man, med, mellan, men, menar, mer, mera, mig, min, mina, mitt, mittemot, mot, ned, nederst, nedersta, nedre, nej, ner, ni, nio, nionde, nittio, nittionde, nitton, nittonde, nog, noll, nr, nu, nummer, nya, när, nästa, någon, någonting, något, några, nån, nåt, och, också, om, oss, på, rakt, redan, sade, sagt, samt, sedan, sen, ser, sex, sextio, sextionde, sexton, sextonde, sig, sin, sina, sist, sista, siste, sitt, sitta, sju, sjunde, sjuttio, sjuttionde, sjutton, sjuttonde, själv, sjätte, ska, skall, skriver, skulle, slutligen, snart, som, står, säga, säger, sätt, så, sådan, sådana, sådant, ta, tack, tar, till, tills, tio, tionde, tjugo, tjugoen, tjugoett, tjugonde, tjugotre, tjugotvå, tjungo, tog, tolfte, tolv, tre, tredje, trettio, trettionde, tretton, trettonde, tro, tror, två, tvåhundra, under, upp, ur, ut, utan, utanför, ute, vad, var, vara, varför, varifrån, varit, varje, varken, vars, varsågod, vart, vem, vems, verkligen, vet, vi, vid, vidare, vilka, vilkas, vilken, vilket, vill, visst, väl, värre, vår, våra, vårt, än, ändå, ännu, är, även, året, åt, åtminstone, åtta, åttio, åttionde, åttonde, över, övre)
>     stopwordList: Array[String] = Array(bara, lite, finns, vill, samt, inga, även, finns, ganska, också, igen, just, that, with, http, jpg, kanske, tycker, gillar, bra, 000, måste, tjej, tjejer, tjejen, tjejerna, kvinna, kvinnor, kille, killar, killen, män, rätt, män, com, and, html, många, aldrig, www, mpg, avi, wmv, riktigt, känner, väldigt, font, size, mms, 2008, 2009, flashback, familjeliv, word, all, alla, allas, allt, alltså, andra, andras, annan, annat, artonde, artonn, att, av, bakom, bara, bland, blev, bli, blir, blivit, båda, bådas, både, dag, dagar, dagarna, dagen, de, del, delen, dem, den, denna, deras, dess, dessa, det, detta, dig, din, dina, dit, ditt, dock, dom, du, där, därför, då, efter, eftersom, ej, elfte, eller, elva, en, er, era, ert, ett, fall, fanns, fast, fem, femte, femtio, femtionde, femton, femtonde, fick, fin, finnas, finns, fjorton, fjortonde, fjärde, fler, flera, flesta, fram, framför, från, fyra, fyrtio, fyrtionde, få, får, fått, för, före, förra, första, ge, genom, ger, gick, gjorde, gjort, gälla, gäller, gällt, gärna, gå, gång, går, gått, gör, göra, ha, hade, haft, han, hans, har, hela, heller, hellre, helst, helt, henne, hennes, heter, hit, hjälp, hon, honom, hundra, hundraen, hundraett, hur, här, i, ibland, icke, igen, in, inför, inga, ingen, inget, innan, inne, inom, inte, inuti, ja, jag, ju, jämfört, kan, kom, komma, kommer, kommit, kunde, kunna, kunnat, kvar, kör, legat, ligga, ligger, lägga, man, med, mellan, men, menar, mer, mera, mig, min, mina, mitt, mittemot, mot, ned, nederst, nedersta, nedre, nej, ner, ni, nio, nionde, nittio, nittionde, nitton, nittonde, nog, noll, nr, nu, nummer, nya, när, nästa, någon, någonting, något, några, nån, nåt, och, också, om, oss, på, rakt, redan, sade, sagt, samt, sedan, sen, ser, sex, sextio, sextionde, sexton, sextonde, sig, sin, sina, sist, sista, siste, sitt, sitta, sju, sjunde, sjuttio, sjuttionde, sjutton, sjuttonde, själv, sjätte, ska, skall, skriver, skulle, slutligen, snart, som, står, säga, säger, sätt, så, sådan, sådana, sådant, ta, tack, tar, till, tills, tio, tionde, tjugo, tjugoen, tjugoett, tjugonde, tjugotre, tjugotvå, tjungo, tog, tolfte, tolv, tre, tredje, trettio, trettionde, tretton, trettonde, tro, tror, två, tvåhundra, under, upp, ur, ut, utan, utanför, ute, vad, var, vara, varför, varifrån, varit, varje, varken, vars, varsågod, vart, vem, vems, verkligen, vet, vi, vid, vidare, vilka, vilkas, vilken, vilket, vill, visst, väl, värre, vår, våra, vårt, än, ändå, ännu, är, även, året, åt, åtminstone, åtta, åttio, åttionde, åttonde, över, övre, och, det, att, i, en, jag, hon, som, han, på, den, med, var, sig, för, så, till, är, men, ett, om, hade, de, av, icke, mig, du, henne, då, sin, nu, har, inte, hans, honom, skulle, hennes, där, min, man, ej, vid, kunde, något, från, ut, när, efter, upp, vi, dem, vara, vad, över, än, dig, kan, sina, här, ha, mot, alla, under, någon, eller, allt, mycket, sedan, ju, denna, själv, detta, åt, utan, varit, hur, ingen, mitt, ni, bli, blev, oss, din, dessa, några, deras, blir, mina, samma, vilken, er, sådan, vår, blivit, dess, inom, mellan, sådant, varför, varje, vilka, ditt, vem, vilket, sitta, sådana, vart, dina, vars, vårt, våra, ert, era, vilkas)
>     remover: org.apache.spark.ml.feature.StopWordsRemover = StopWordsRemover: uid=stopWords_1049f7ff5bc5, numStopWords=497, locale=en, caseSensitive=false

In [ ]:
val removed_df = remover.transform(tokenized_df).select("c", "filtered")

  

>     removed_df: org.apache.spark.sql.DataFrame = [c: int, filtered: array<string>]

In [ ]:
val vectorizerall = new CountVectorizer()
   .setInputCol("filtered")
   .setOutputCol("features")
   .setMinDF(5)
   .fit(removed_df) // returns CountVectorizerModel

val vectorizer = new CountVectorizer()
   .setInputCol("filtered")
   .setOutputCol("features")
   .setVocabSize(1000) // Size of dictonary
   .setMinDF(5)
   .fit(removed_df) // returns CountVectorizerModel


  

>     vectorizerall: org.apache.spark.ml.feature.CountVectorizerModel = CountVectorizerModel: uid=cntVec_08c9e7fcb212, vocabularySize=128728
>     vectorizer: org.apache.spark.ml.feature.CountVectorizerModel = CountVectorizerModel: uid=cntVec_b85ee1ac7a89, vocabularySize=1000

In [ ]:
vectorizer.vocabulary

  

>     res8: Array[String] = Array(alltid, känns, bättre, faktiskt, verkar, problem, behöver, skrev, olika, precis, svårt, fråga, håller, saker, låter, förhållande, brukar, förstår, tänker, borde, älskar, varandra, skönt, tillsammans, tänka, partner, vissa, länge, ville, vilja, klart, känna, enkelt, annars, sambo, egentligen, människor, prata, hålla, tråden, mindre, istället, direkt, hoppas, knulla, längre, säkert, anonym, jävla, liten, följande, tänder, dessutom, handlar, nästan, börjar, hitta, själva, absolut, stora, liksom, orgasm, däremot, bästa, större, sitter, tyvärr, fortfarande, funkar, sängen, person, tillbaka, tidigare, otrogen, dåligt, händer, börja, vanligt, massa, livet, bröst, inlägg, kolla, först, hemma, berätta, långt, undrar, mannen, försöka, kropp, beror, började, testa, oftast, kuken, försöker, bilder, frågan, speciellt, trots, självklart, sluta, liknande, känslor, enligt, gravid, konstigt, tycka, sidan, tyckte, ifrån, början, tänkte, spelar, problemet, gånger, relation, kvinnan, trodde, gången, lycka, sexet, förstå, analsex, ....., pratar, jobbigt, anser, äldre, snygg, använda, sjukt, kondom, sexuella, sätta, oavsett, otrohet, vänner, flickvän, sexuellt, pengar, fitta, visar, skriva, igång, skillnad, kroppen, använder, träffa, kände, lever, intresserad, sexliv, ungefär, titta, stort, intressant, fattar, prova, kärlek, erfarenhet, normalt, hittar, oralsex, kompis, munnen, fungerar, snarare, personen, väljer, jävligt, otroligt, extra, träffat, skull, svara, runka, enbart, slicka, efteråt, svenska, tagit, tanken, särskilt, människa, pojkvän, viktigt, exakt, samlag, personer, knullar, ingenting, yngre, välja, betyder, säker, övrigt, tanke, knappast, anledning, tittar, lättare, medan, lägger, mamma, sugen, visste, såklart, vågar, föredrar, endast, vänta, ålder, tydligen, ollon, barnen, tänkt, saken, kontakt, snabbt, möjligt, minst, fortsätta, ensam, behov, behöva, filmer, frågar, gammal, äckligt, dålig, jobbet, lämna, givetvis, snygga, fittan, huvudet, hjälpa, vanliga, skaffa, lilla, kollar, igenom, sällan, försök, träffar, njuta, antar, orkar, stämmer, suger, personligen, förklara, sätter, världen, förmodligen, söker, oskuld, krogen, skrivit, ifall, exempel, roligt, nätet, förhållandet, situation, ställa, träffade, alldeles, trevligt, jobbar, antingen, antagligen, njutning, ansiktet, brudar, sexuell, tankar, testat, gamla, nyfiken, barnet, knappt, extremt, läser, frågade, tråkigt, kändes, riktig, fantasier, njuter, klarar, trött, underbart, upphetsande, hände, räcker, sexigt, senare, totalt, iallafall, börjat, ställer, ordentligt, pappa, förhållanden, fullt, passar, sperma, vanlig, vägen, förut, runkar, pratat, känslan, kalla, tiden, minns, förutom, sättet, ledsen, tvärtom, någonsin, känsla, hända, filmen, tillräckligt, innebär, röven, plötsligt, önskar, risken, penis, aning, naturligt, p-piller, försökt, kring, härligt, värsta, jobba, chans, sådär, benen, annorlunda, sexualitet, hahaha, föräldrar, mesta, massor, spännande, bilden, hittade, magen, naturligtvis, provat, resten, erfarenheter, kompisar, luktar, hjälper, handen, ärligt, frågor, trådar, trevlig, saknar, utseende, situationen, helvete, betala, sexlust, intresse, läste, acceptera, dildo, sambon, ansvar, normal, seriöst, hittat, brösten, naken, glidmedel, köper, perfekt, snacka, lusten, förstås, största, gilla, kukar, slipper, delar, fundera, märker, medans, åsikter, litet, långa, betydligt, lyckas, slutade, fantasi, snälla, kläder, sexig, familj, velat, stånd, internet, ovanligt, onanerar, rumpan, kännas, beroende, slags, jätte, självförtroende, såhär, forum, bakifrån, princip, hänger, t.ex., definitivt, funderar, sidor, polare, slutar, försökte, klitoris, upphetsad, svarar, plats, männen, partners, killarna, beteende, menade, talar, tända, åsikt, onanera, verkligheten, svaret, öppet, lugnt, avtändande, väntar, berättar, rumpa, antal, ärlig, träffas, umgås, nuvarande, fingrar, ämnet, vuxna, borta, bilderna, spruta, svårare, riktiga, trekant, alternativ, berättade, orolig, älska, dotter, prostitution, klara, respekt, ......, överhuvudtaget, kvinnorna, osäker, grund, relationer, satte, diskutera, släppa, funka, minsta, pinsamt, svensk, gifta, pratade, skiter, fallet, anledningen, någonstans, orgasmer, slickar, öppna, porrfilm, skicka, fortsätter, spela, omöjligt, uppenbarligen, förspel, grejen, ställe, allra, öppen, sover, visade, våldtäkt, stället, generellt, närhet, håret, finner, grymt, smeka, stoppa, omkring, rejält, sexlivet, upplever, lagom, slappna, kallar, oerhört, talat, skillnaden, prostituerade, vacker, inser, homosexuella, tackar, samhället, sälja, skönare, storlek, ändra, ryggen, snackar, likadant, tvungen, partnern, sexiga, vatten, avsugning, ställen, gillade, hatar, åldern, initiativ, stark, krävs, chansen, dåliga, farligt, huvud, därmed, sämre, trycka, tydligt, ansikte, knullat, trist, fysiskt, beskriver, förresten, gratis, ringa, trosor, onani, hållet, underbar, lyckats, snyggt, gränsen, slippa, allvar, faktum, uppskattar, relationen, träna, bestämma, månad, förslag, världens, träffades, himla, räknas, bögar, snäll, rätta, ordet, hemskt, ögonen, leder, duschen, vuxen, mysigt, kvinnliga, horor, porren, kräver, närheten, förstått, datorn, nämligen, manliga, mensen, fruktansvärt, högre, rakar, behövs, utlösning, klipp, lider, skada, funderat, fantiserar, fungera, sanningen, desto, ångrar, normala, längden, trivs, svarade, främst, säljer, storleken, ragga, erkänna, allmänt, varann, vackra, kallas, älskare, oftare, ursäkt, råkar, finger, svart, märkt, läkare, otrogna, stanna, tillägga, månader, bruden, fatta, undra, piller, smakar, andre, senaste, värld, överens, ställningar, medel, lycklig, bestämt, soffan, bestämmer, dominant, analt, frågat, försvinner, faller, hamnar, räkna, skäms, flickor, rolig, taget, löser, upplevt, dagens, skolan, starkt, betalar, jäkla, tacksam, kostar, attraktiv, påstå, ljuger, slutat, sorts, kollade, lyssna, oskulden, lämnar, herregud, uppleva, positivt, nakna, dörren, angående, forumet, alkohol, bekräftelse, smeker, ollonet, undan, knull, reagerar, personlighet, halsen, ställning, medveten, överallt, smärta, stackars, knullade, troligen, typen, enormt, påstår, visserligen, inlägget, jättebra, konstig, djupt, undvika, tvätta, satsen, körde, flytta, övergrepp, relativt, kroppar, fantastiskt, obehagligt, missionären, äcklig, passa, hormoner, tidigt, kunder, älskade, allvarligt, garanterat, utifrån, snopp, berättat, exempelvis, läggning, bredvid, mening, sjuka, seriös, fullständigt, hårdare, massage, använt, länder, ångest, samvete, fingrarna, följa, flicka, letar, skiten, uppmärksamhet, använd, negativt, grattis, tvinga, beslut, historia, sorry, diverse, kommentarer, sprutar, tunga, betalt, förbi, argument, svarta, hinner, upplevelse, valde, akten, därifrån, trycker, förtjänar, rummet, lovar, diskussion, gångerna, pungen, påverkar, märkligt, kollat, underlivet, attraherad, könet, fantisera, besviken, betydelse, förhoppningsvis, tillfälle, hänga, graviditet, fötter, smaken, förvånad, uppfattning, skylla, förstod, misstag, svamp, olagligt, tyckt, detsamma, smisk, läget, inställning, viktigaste, tillfället, smala, glömma, hantera, självkänsla, lagen, funkade, ursäkta, skämt, duger, framförallt, tungan, muskler, stycken, spiral, nervös, naturliga, isåfall, orgasmen, jämföra, starka, idiot, verkade, människan, beter, samband, möjlighet, majoriteten, händerna, viagra, psykiskt, intresserade, ringer, meningen, roliga, allting, kondomer, uppenbart, bryta, tjock, slidan, attraktion, sönder, länken, förrän, ljuga, vafan, skydd, dricka, köpte, porrfilmer, närmare, förlossningen, lösning, talas, försiktigt, slapp, korta, samhälle, analen, svartsjuk, handla, troligtvis, låtsas, homosexualitet, skratta, smart, förlåt, omkrets, pengarna, polisen, slutet, finna, vidrigt, hårda, äckliga, länkar, skapa, utmärkt, lyckades, anses, numera, varenda, damer, gissa, besök, stolt, högst, resultat, privat, härlig, rädda, sköna, visat, underkläder, skiljer, fittor, tecken, manlig, litar, behövde, poäng, vecka, undantag, följer, syster, anledningar, uppskatta, snyggare, trevliga, sexlusten, skämmas, ringde, styra, längd, tillstånd, vänder, viktig, kopparspiral, situationer, skriv, tillhör, skrattar, kärleken, tappar, kvinnors, snubbe, hörde, kvinnans, därefter, svälja, denne, starta, tittade, ytterligare, snoppen, bevis, kvinnlig, vårat, dumpa, former, gränser, vända, galen, pröva, graviditeten, tråkig, maken, vägrar, kläderna, möjligen, förlora, sårad, äntligen, slickad, underbara, vaknar, lagligt, rekommendera, trång, övriga, allmänhet, topic, fakta, beredd, hjärnan, mannens, tjänster, bodde, hoppa, känslig, bjuda, märkte, tuttar, handlade, ladda, gissar, använde, speciell, kåthet)

In [ ]:
// Count the word frequencies
val tf = vectorizer.transform(removed_df.select("c", "filtered")).select("c", "features").cache()

tf.take(1).foreach(println)


  

>     [0,(1000,[],[])]
>     tf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [c: int, features: vector]

In [ ]:
//Train test split
val random_order = tf.orderBy(rand())
val splits = random_order.randomSplit(Array(0.8, 0.2), seed = 1337)
val training = splits(0)
val test = splits(1)
println(training.count())
println(test.count())

  

>     90818
>     22424
>     random_order: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [c: int, features: vector]
>     splits: Array[org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]] = Array([c: int, features: vector], [c: int, features: vector])
>     training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [c: int, features: vector]
>     test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [c: int, features: vector]

In [ ]:
//Logistic regression
val lr = new LogisticRegression()
  .setLabelCol("c")
  .setMaxIter(100)
  .setRegParam(0.0001)
  .setElasticNetParam(0.5)

// Fit the model
val lrModel = lr.fit(training)

// Print the coefficients and intercept for logistic regression
println(s"Coefficients: ${lrModel.coefficients} Intercept: ${lrModel.intercept}")



  

>     Coefficients: [0.012034881077738254,0.054749123247116956,-0.018670288519326856,-7.021830440064947E-4,-0.04984601448023173,-0.01960659399608072,0.06788633652970974,0.3658607943208939,0.11659653084909366,0.05604079370814291,0.07685425274985103,-0.008468025230912052,0.0391701646500979,-0.033515243276451716,-0.06493448182753382,-0.03701721186127038,0.011268851727329764,0.1449311554107987,0.06065879377418217,-0.15069124307737897,0.10915326282494242,0.06783077656246114,0.05969565370297491,-0.020769459498832972,-0.06261778492945069,0.08577522433581207,-0.006862324165650049,0.03782859423666832,0.06198524086180862,0.13908268245099475,0.007844162996551806,0.025652010132909193,-0.18912631254511755,-0.10867825252819427,0.8664265850498226,-0.02547947814133745,0.057530635888478586,0.10492674910822022,-0.03847367553271068,-0.14881327492331717,-0.022845699160626877,-0.06781204614590816,-0.04021335266707723,0.21566788045122323,-0.29173485590070736,0.04495454874543587,0.0294091172498715,1.9102192494046342,-0.7434831809028792,-0.03180889178073911,0.3259509612753385,-0.017420204360090747,-0.09291550066871183,-0.009700089772085401,-0.021423869644104764,-0.0952920192866336,0.04372590555328903,-0.05182709211199029,0.4054095751765017,-0.0015418384743616678,0.30166939871637716,-0.008625628823386546,-0.01301034526932088,-0.09346757976062901,0.05588368226254467,0.01909514624581868,0.18174302569463333,-0.08605671976445488,-0.015753827573480195,-5.601965641818634E-4,-0.023349995148924753,0.08208282791642292,-0.013611169981826302,0.1866299749047193,0.04651553204022181,-0.02243740751190066,-0.041923815604583345,0.04400362440626569,0.033972156032699256,0.0338895759109998,-0.025341977351488403,-0.31351670875909365,-0.010163927695973273,0.09794487111496217,0.20670649833071755,0.1576223634136063,0.08472981814893138,0.2058091384241183,0.2657519574375513,-0.10501552857371316,-0.09641074571519659,-0.07172194042386791,0.016222185108798942,-0.008595385028257917,0.00790372837572551,-0.27707875133303456,0.0029916497470590915,-0.2008495658807781,0.034459186642024846,-0.23870100032669248,0.04945331993674277,-0.02535531471201606,-0.018476968217059218,-0.24049925909032985,-0.0442115247820918,-0.18006183563733016,0.41468352026988864,0.0826128565045806,0.22721538736671842,-0.19455130390068057,0.07924648925853262,-0.12057240856511867,8.796749758057982E-4,0.029223345211943922,0.056738345869352835,-0.12492899160741508,-0.07065799553669991,0.14269466083031157,-0.021577327023168582,0.2708425893101856,0.022718750379068987,0.31298719482562914,0.04174758131773422,-0.024289552171616175,0.11538591012229792,0.3031719044579777,0.12376264292335062,0.04871709865107555,0.03576252569747173,0.04209385028919935,-0.03516373445406039,0.1352254301854727,-0.030463231759041757,-0.0351336271690597,-0.0474222886082609,-0.05073364856526572,0.448165373053336,0.09834286254762138,0.1898609538216264,-0.588243076444115,-0.010448460477741213,-0.02289293500811144,-0.0445240518269938,0.10202492427690375,0.29133553390831135,-0.008169425704268804,0.1263799624993872,0.040075493381239655,0.1182182848219929,0.06348202930428751,0.053203006193442336,0.3511860664606915,0.055773972155107564,0.3155361796257535,-0.09490325737146313,0.13065533165023197,-0.03005212554964348,-0.13293637408696748,0.2964807739464729,0.030066946027573416,-0.050680449885978536,-0.16304745084691014,0.058937704292303016,-0.08729881847381304,0.05044857300267097,0.0707554774217764,0.13604594147358545,-0.13069782402600624,-0.07154816286454299,0.0,0.2917815232384764,-1.093689729083599,0.0553507205089596,0.01139643465753628,0.015120957271103639,0.06813385844874861,0.04387820025840367,-0.24134702236249453,-0.004272164957732482,-0.00748541172127404,0.04503569694623818,-0.08722581695452088,0.1622284213815299,-0.1234731472933675,-0.205246290027284,0.025778260648620974,-0.027953314842683222,0.033200487571370195,0.04505891673706167,0.11994813810854543,-9.913506640536879E-5,-0.18202317443698707,0.033235913199631595,0.05268157388150551,0.31020843536802434,-0.01983620215014756,-0.07788233658118347,-0.14736382017768956,-0.04736163649070417,-0.3451185371230663,-0.16649507838884597,0.22031085733204134,0.07769084840998178,-0.0025441170483045824,0.0900482497833232,0.015435207778056735,0.2721225986628336,0.11531788113175638,0.093833095221353,0.1740843950631458,-0.004894942436691474,-0.17996170659816949,0.1847105714170813,0.04515253700021516,-0.04325958296961873,-0.2624009720199595,0.29290129675713045,0.06899454389842176,-0.22183236270921983,0.07384376249888457,-0.018846159626533934,-0.17490713979409594,-0.14988061595373114,0.10879703619324278,0.07016867271326864,0.02157522531545984,0.13185265290513246,-0.36024014215131833,0.0908224774976646,-0.10156656502440271,0.061449255170177144,-0.032057071081892874,0.1505309080724123,0.20842519922389402,-0.3132087942577477,-0.007584472324997918,-0.3257100490807896,4.42460018528334E-4,-0.00780251133120894,6.22578319313422E-4,-0.1124957400969359,-0.2795545985509358,-0.08451511754281421,-0.08075359144998275,0.12494149856987467,-0.19170637077555475,-0.05623257469733826,-0.013854582713781363,-0.2507537945869827,0.0013190630845595654,-0.1537362017063143,-0.020052532159523916,-0.2975200655833895,-0.13635537870854117,-0.06466779187419254,0.08223096915225866,-0.5175098929001537,-0.2633814436163761,-0.011450387159361912,0.06538580530797371,0.19314630257306,-0.3105228235543972,-0.32039956860680663,-0.009329439459293456,0.17179501209134562,-0.23311820333877656,-0.19521437494393978,0.05998962576650433,0.2572850475363575,0.23625063877660657,-0.15457244890110536,0.0923697934933707,-0.12998487787140448,-0.4249159027289643,0.056667807271936564,-0.004865411736312165,-1.4341202867987122,-0.027364191787892927,0.018231618804159674,0.046952823438036306,-0.29783865673521076,0.4871219410854013,-0.1426379378457232,0.18844771967012014,-0.1032460431713518,0.1626090354951455,-0.0055996416924629115,0.022124990577547847,0.0941686148882238,-0.1218307434805607,-1.948023713046166E-4,0.20021010581910434,0.052409754361748585,0.06464831489759397,0.34798921073122885,-0.0015266555599261562,0.12558956330180218,0.16070221349533886,0.06886236421382227,-0.29648654518715484,0.08786851684028181,-0.21248271959094256,0.30608752008802165,-0.07730882824927507,0.0027146146269781423,0.07473332946819151,-0.3240151830226001,-0.06391513624488988,0.13817666757688418,-0.04530350063989395,0.1297231951933472,-0.051728131713487384,-0.03321433649854208,-0.04128660285041306,0.23266411514860097,-0.15289008707497959,0.17849348823070252,-0.06970101800062634,-0.17218043856645748,-0.07449487310094538,0.014195657477038942,0.2606871834489373,-0.04609794225809198,-0.002052049889426422,-0.13620112016878036,0.013306498967766523,-0.42191553005004234,0.005844279742691243,0.02076444960542404,-0.4933931588960152,-0.038422919972909024,0.2356120202227697,-0.06579469337821374,-0.051012546929538494,0.005331848284929072,-0.09221023737754579,-0.06087267461854112,0.23704880935849715,-0.009656263535655355,0.4921888058524759,-0.01087438046100034,-0.01502844806410026,-0.07607891292283595,0.03817476957852833,-0.009804010387257414,0.03795906350849129,-0.35676902249924103,0.33591434981158885,-0.16606830786241042,-0.14595690014944043,0.3315523629657571,0.23402527397653028,-0.1498650076286213,-0.24350692508133717,0.22892096399251047,-0.4061138710228588,0.17771534055464983,-0.15540719009539355,-0.049781431901548406,-0.04354176967343981,-0.03821811544556826,0.12039462581785529,0.0102961254535218,0.42268714360577114,-0.008425442567187792,0.3261278345769137,-0.20164808987386101,0.1936009405505642,0.13495933868815632,-0.2153309291097306,-0.7531581204972304,-0.08189905738280281,0.1615347650172357,-0.0704598584105252,-0.2234652892617414,0.03262228981069153,0.024468942270439394,0.9913337832941463,-0.12751538186794192,-0.12432378769596543,-0.18715739445837418,-0.15131207386703638,-0.0396690533997529,0.053115069921639994,0.06209255508093185,0.03229281047676171,-4.1236124422019645E-4,-0.4563328317264295,0.26922316856246503,0.05267509142092671,-0.1112290774648626,0.022701720043101507,0.08073305332789317,-0.15146092414790632,-0.029756072366294718,0.23937074459294055,-0.08927691128409107,-0.4319529251618577,0.18148419073607236,-0.12341964169927175,6.105362551971156E-4,0.06505401968343094,-0.09089987552627256,0.19839742780143926,0.09034625989775544,0.37423785506973417,0.07957267283918446,-0.04222881699669273,0.15174057668736302,0.22385051092014271,0.05751924715014821,-0.2681044307324861,-0.04243165861037261,0.04871357912233753,0.24028410625846203,-0.1708930012707026,0.01804083280528032,-0.2041028793957883,0.6742217317608472,-0.03146694711961978,0.08393199912690932,-0.2662352332336272,0.23112569839233965,-0.23100541263273777,0.20423467534752376,-0.37516490221627313,-0.015128918505479219,0.32890687654643713,0.06402771567372638,-1.223628085683817,-0.07237774462682872,-0.017146548545342675,0.06992260064394175,0.03812325669637177,0.02519177867021323,-0.050254449221855825,0.4054285195915163,0.25942425697672905,0.2631501656415759,0.08057323971893782,-0.1187289981976431,-0.23839063272045255,0.03155903911784348,0.08448948964322862,0.13251915542697446,0.10809299879153651,-0.03767763560216509,0.17387609343067872,-0.260405970633715,0.19787948105428207,0.15143970106348417,-0.05940336671305686,0.06605067055004443,-0.18670188607111252,0.2670609267066871,-0.021706233778829385,-0.13859258698124832,0.11112616220762202,-0.05976993055167087,-0.6225735377403595,0.106509815288614,0.18974196670911833,-0.2567380660701016,0.004770176305522026,0.04754704822084308,-0.1699424559096812,0.0402895676618791,0.0777726017845089,0.038081842140552546,0.5561088029971352,0.16884116128386326,0.08191114957422398,-0.045057666656262894,6.85922483397264E-4,-0.005910394608905731,0.27038597477582876,-0.2691262240924111,0.14963160787445925,-0.03937379044467165,-0.3196642422071246,0.13177679915638163,0.33561599826845206,-0.15980953888134086,0.07642262101472075,-0.11967173887828605,0.2547355919983294,0.1744374045044375,-0.1861071410133244,0.023429975151031943,0.07131842820037826,-0.33330025723164414,-0.27482782375532855,-0.31206694519397127,-0.11435470099020119,-0.035631038650418174,0.07095932765021497,-0.03903663559775185,0.026584285267603636,0.07444813442476762,0.14703161675402662,-0.16591038744415218,-0.05650706588171715,0.05732747113137313,-0.08487501333259596,-0.20777124241979233,-0.08377081510633792,0.16227680297348973,-0.05238669544246557,0.049172387985959194,0.18718318096440634,0.05540655021071432,-0.18436234854761271,0.024676848377552327,0.13491393516683003,-0.013792583420543611,-0.16162551943902828,-0.21276950106504366,0.01583285302599497,-0.06484797733270185,-0.17497100375274494,-0.20487536613482327,0.00208296958096768,0.06011805050246887,0.005467623902667266,-0.014276872236086105,0.00330355193703288,-0.018124433750214358,-0.3862444709246528,0.1382640912713296,-0.051837238997829924,-0.1655678126720367,-0.17019184421611647,-0.03952157968296064,-0.9982302179709829,0.14761161596515465,0.10778770439559987,-0.03635335315119181,0.20976301410066273,0.054971701201378426,-0.019548940303363117,-0.6521543891383563,0.17363266700189706,-0.20835553209162508,-0.19509043086630565,0.011764805047909709,-0.033670895738190085,-0.23194036566881776,-0.05776631672172416,-0.2168266515185609,0.04317614404510422,-0.24073452470067638,-0.10385640067497234,-0.004863357570454622,-0.0742217403674213,-0.20708948712300682,-0.07703078754317942,0.23755076402287306,0.08152526535619334,0.08494825458472612,0.0028180952762931373,-0.04877321336901508,0.30948011664058656,-0.06545257422130774,-0.23180950227545405,0.33049186246244455,-0.08968766462484981,0.36568211557549607,-0.218388834755498,-0.053743062524313004,-0.002064735744471037,-0.023140541300552295,-0.05450239139368648,-0.00733632699358798,0.5015657986525798,-0.13010871476859948,0.1758325768588694,-0.05126192029999816,0.12011889567940791,0.13938207571239175,-0.304330535931412,-0.008277510940418582,-0.0013861041103585567,0.05735393392603044,0.05084478358945873,0.19410962659578623,0.05671577973750511,0.08630151372642345,0.2271451183723975,0.9075917959122932,0.05987317190591268,-0.6113885507748231,-0.024181085096516217,-0.1556681145905628,0.2624378482356017,0.3466805273019043,-0.12186702018872624,-0.29492456989432464,0.037601548702128666,0.34484710508338307,0.12520663887333647,-0.17402711192549308,-0.5906143760564851,-0.004563485619697921,-0.1660459102288016,-0.04991668182038898,0.06674623772419003,0.14910513817922896,-0.09002541186464134,-0.01722638803821666,0.5540024821253612,0.3243485174786765,0.23874855152051067,0.20509560568750343,-0.06834586270490019,0.06837592899954666,-0.8517113297871225,-0.13746287938869592,-0.003127870515962289,-0.03980045059977697,0.04344293110439681,-0.25010610848592296,0.27305021195814483,-0.24360077030045893,0.08517363163810351,-0.014926480193482318,-0.14607039221072776,0.08785594469247603,0.11401850428993084,-0.14330487820482427,0.04871715486583609,0.06639840072727778,0.05165765222894718,-0.1941203720610051,-0.24088260428834937,0.3375898130375687,0.05881338465209857,-0.18700978251007705,0.32069744247506826,0.23337674034062855,0.09060729916694447,-0.026379468819808828,0.07569408631698583,-0.17660931301526406,-0.15173813862948893,0.19271468192349117,0.09646225554856849,0.06872646694332472,0.3391961257545038,0.337889915298144,-1.059696209936961,-0.02770675621268566,0.11970931701871572,0.08579288049498715,0.001171524747983477,0.002581219205727864,-0.21974119409551898,0.0014679041327962479,0.12876832730883137,0.10084116877046098,0.36468516070996776,0.054908550343025685,0.042256151880877994,0.1813834113747525,0.07844457415245222,-0.023292279775391108,-0.021102678923953188,0.2737552313136419,0.14099264170682013,0.08906832927864838,0.20547179673510704,0.05349793422100516,0.07502926725384039,-0.37627693701684917,-0.0010290853759099686,0.00993683214257727,-0.11349155237934572,0.011343211673141423,-0.23664679647104714,-0.06599131349709456,-0.22150692068644473,-0.18064760493105433,0.1274954409462321,0.08338658593067566,-0.06388220592766536,0.3228887060397382,0.1151433245864652,0.06702483800327158,0.14684039287974768,-0.36729424741680977,0.16951032530752344,-0.005622669679589971,-0.013103134199721938,0.06080168249765485,0.1928110443718361,-0.02062039902160816,-0.05375078812433159,0.11452511488079761,-6.88887677943987E-4,-0.5149795496467623,-0.6704386668482576,-0.12245839226459948,-0.00796491922398551,0.04920003264019381,0.021789847872863248,3.7486424850170465E-4,-0.25971838182961593,0.27152057336697377,0.16209121577224364,0.022727287826024383,0.05358200577100021,0.062388517692628466,0.13198901854668313,0.0012215479150775733,0.025215743555664215,-0.6562932993469595,0.12513977599008058,0.03198322122948516,-0.1396994263995953,0.16777189603608558,-0.23858781735539286,-0.22229658699957336,0.687661002486799,0.1190107554958362,-0.13012811210468514,-0.022248124591068695,-0.01808834128255245,-0.10558237922387449,-0.2515532191927218,-0.2786735839398049,-0.19939332341584534,-0.3027783099924844,0.08119268186310456,0.33774564054646317,0.06878138522528211,-0.19224258321209753,0.05420665563863204,-0.25998193588407714,0.031569955678304014,0.11053032690262808,-0.47586739781955356,-0.03158236611658082,-0.10632718167174073,-0.2713033592757659,0.4450764302783433,0.31053084990555924,0.24552581615238966,-0.13810676629974208,-0.028889382926887974,-0.11868515288071713,-0.058105656820407095,-0.48366223064064495,-0.225724666175751,-0.027426654990540023,-0.1112011496252026,-0.05127821909957626,0.2836746272337936,-0.06633557878702641,-0.1280825643627053,-0.015000591793654915,0.004223266753493742,0.02518149345501679,-0.4895996077747331,-0.19603926774810898,-0.49891892253683207,-0.03211503454304652,-0.16268186727981707,0.10736161317695067,-0.1388004438579746,-0.011793969361071473,-0.27135426523511946,0.02344396583782779,-0.006088466685735631,-0.6583778562433434,0.3001427337828096,0.1195800009417879,-0.031298065106802664,-0.018476857345190473,-0.004901261284260017,-0.02548034894202024,-0.044605604925384804,0.1201707973097875,-0.11789965564921147,0.3445863807564719,-0.46542299035208423,0.027819499156462004,-0.11168466630068007,0.30158514913134105,-0.1744454449557201,0.021305173310483753,-0.10234694151344824,-0.1571585147706423,-0.025516377032657144,0.06623664107435272,0.17104423780355413,-0.0029005130851074224,0.735394982422738,0.06070176322940083,-0.2306811143266667,0.006478145270742422,-0.03459519030951447,0.10602997011975472,-0.14916514934364286,-0.23591128763467,-0.031479395158515354,0.09259629996491402,-0.11526725268530479,-0.007140783670354588,-0.07017294776471977,0.18584164835761474,0.29784178120014754,-0.03637415457070874,0.14802709507572967,0.36291329155384466,0.15657119245387457,0.19148148215261643,0.11792855011853534,0.05783795232684082,-0.08271231636613452,0.1797271821895709,-0.19739572440987152,-0.39674625399258046,0.19386736959713835,-0.06115475669537964,0.08920149957984232,0.28033825956222286,0.09096128276587337,0.12264598936049949,0.18268538631956024,0.07580219873331681,0.05259461987827397,0.19302821657875224,-0.07173340395864584,0.02327405522053611,0.05044169422560377,0.5057102883967028,-0.15681990863006862,-0.020946548402909243,-0.053060613965376924,-0.08086160782880382,0.15086633771687108,0.07705459392547466,-0.22262681593131523,-0.275096744246792,-0.22294295768248443,0.1621649586427284,-0.06796274807771206,0.186267923833034,-0.09852586458467766,-0.012471078808955824,-0.12635212097496043,-0.278942777036747,0.29833664778853564,-0.2871640526223206,-0.0958654566324368,0.07415761893391996,-0.4365011186342986,-0.08504994248392293,-0.024510579392829676,-0.18948179193117926,0.1351648802272352,0.27611068472760986,0.3267294645725186,-0.012917952836200664,-0.739949575748971,0.08383451311542393,-0.18724352563709953,-1.5121327707986945,0.46412547150394956,-0.09252887284647938,0.3425315923684792,0.009993714131011978,-0.07903587886567183,1.2777080229192614,0.04681279815976317,-0.12549614129721862,-0.019603937114478144,0.08044848532481505,0.02067050083511978,0.23536016695093617,0.25164070435522135,0.15286945915868635,0.07351359106078433,-0.3732943578401295,-0.09486861209962394,0.00394205546942895,4.0280896956812153E-4,-0.060955528707448456,0.11417012899752621,0.22280587925077014,-0.19998372087535127,-0.16695946545796858,-0.17488901449418146,-0.28421395684231626,-0.0017724321252448895,0.06629805872298522,-0.14175034493129612,-0.9675842944309819,-0.09073119136826513,-0.3736454644838643,-0.1197357833309627,0.20065472390783076,-0.24146526472756594,-0.12753015320120628,-0.7777046708739789,-0.20655834769725,-0.538520028690995,0.03289565384594195,0.01922681351543435,0.0035088438159594088,-0.0399432276131162,0.26445308711403553,-0.040664816237250566,-0.009914363081479012,0.16845082378038112,0.15956934186592026,-0.00801593003007189,0.03376174885555629,-0.059983869066039745,0.15319799878208784,-0.18692435070964655,0.014473269487791051,-0.07605937001734993,0.1221827125486563,-0.08136078124908643,-0.07024225749797054,0.09586571462325813,-0.20964153119581247,-0.3030169442074872,-0.04019294000313479,-0.0783302486437824,-0.2783218384637299,0.06711881425540901,0.030225722930565194,-0.03632584492407763,0.13708504358272774,-0.10712475224262256,-0.015056868027755478,0.11030768293105225,0.7860354904664587,-0.11056941789610546,0.07388647004416231,0.12771007696016573,0.27502634150231636,-0.030154625106778278,-0.0641548121988506,0.2533198831930802,-0.9589299979769175,0.11089947644356098,-0.4098771057932954,0.06653920319792248,1.6315426650526292E-4,-0.23403109105157138,0.1255002967134638,0.2919266412707532,-0.23189371251629343,0.22500214980443953,0.017449674898406837,-0.16322781300385952,-0.033030542327454906,-0.13541772984792308,-0.1577457757469299,-0.1851581532832446,0.2456535721664056,0.2026261965065334,-0.3841433386565329,0.5699326865737413,-0.14281269128007068,0.9512103966721316,-0.10455252357215059,-0.17097598112169193,0.07683253273293977,-0.05111665260523699,0.29825621726728635,0.07407068113532053,0.13881704139655482,0.27256240400550663,-0.023656477621422298,0.061326384371968135,-0.12293100026223018,-0.045609140420701175,-0.2932195080470082,-0.05932334878638516,-2.000393280148797,0.07609227684546657,-0.03578534125365624,0.009462686293127963,0.0,-0.2673447331177513,-0.09924898268528683,0.16770986598714427,0.12821856422485256,-0.04389261023562544,0.08884810660797507,-0.18557795155311554,-0.0747183901758229,-0.6762518020052284,-0.17162788488175065,-0.026389975613932252,-0.18211835713630545,-0.19550508030820724] Intercept: -0.17609510341888768
>     lr: org.apache.spark.ml.classification.LogisticRegression = logreg_08ebd737e5b5
>     lrModel: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid=logreg_08ebd737e5b5, numClasses=2, numFeatures=1000

In [ ]:
lrModel.binarySummary.objectiveHistory.foreach(loss => println(loss))

  

>     0.6931469164930032
>     0.6697068350130967
>     0.6040162961293869
>     0.5643872426258156
>     0.538658126714753
>     0.5111035646192438
>     0.4845550511584882
>     0.4605931356197869
>     0.4440462195676818
>     0.4308344486613433
>     0.41973686285798256
>     0.4116742096526425
>     0.4037417415176244
>     0.3981183576550698
>     0.3935972692200076
>     0.389502197006037
>     0.38588114726194356
>     0.3821208173404888
>     0.37750164408734965
>     0.3735331950245333
>     0.37198437456899763
>     0.37015467526269996
>     0.3690864548576534
>     0.3675627194590183
>     0.36592575119200793
>     0.36418451217286235
>     0.36384664945024786
>     0.36313612442243887
>     0.3626992008778885
>     0.36222570036563656
>     0.36179316835449155
>     0.36148058521430415
>     0.3612026651293251
>     0.3606571813768764
>     0.3604605186455255
>     0.3600413623492421
>     0.3599280228552643
>     0.35957068780407836
>     0.35936454376206756
>     0.3592185908540887
>     0.35922892408364476
>     0.3587116607785645
>     0.3586807242299566
>     0.3583492316857075
>     0.3583299655343311
>     0.3581156157737276
>     0.3580650311785918
>     0.35790200512850073
>     0.3578318777906233
>     0.35769754686891236
>     0.35764665939863216
>     0.3575480265943723
>     0.3574934433497115
>     0.3573846785818065
>     0.357321511053603
>     0.35722827191595274
>     0.3571773382002145
>     0.3571047563906147
>     0.3570501310193374
>     0.35697191834244657
>     0.3569148973020041
>     0.35685450301158717
>     0.3568134870896976
>     0.35675510527818427
>     0.35671416184709925
>     0.3566518332785649
>     0.3566082958202466
>     0.356554410760951
>     0.35651942758850214
>     0.35646675071526984
>     0.35642744765441353
>     0.35638034534018725
>     0.3563381757079962
>     0.3562963068557861
>     0.3562627699981137
>     0.35622182795037916
>     0.3561906870711827
>     0.3561418249003773
>     0.35613521231638573
>     0.35608228220677735
>     0.3560561337572628
>     0.3559946045748659
>     0.35597979391063
>     0.35593377332555254
>     0.3559200964269027
>     0.35587895533808883
>     0.35586119452291465
>     0.35583146444261377
>     0.3558178165861431
>     0.35579201332782984
>     0.35578086970080497
>     0.3557506853129291
>     0.3557369542846441
>     0.35570765391819237
>     0.35569864824865544
>     0.3556736411763023
>     0.3556611756585955
>     0.3556385132645451
>     0.3556240383854611
>     0.35560627758044827
>     0.3555938098859026

In [ ]:
//Ugly code to lookup maximum and minimum values
var maxarray = Array.ofDim[Double](5,2)
def findmax(idx: Int, value: Double) = {
  if (value > maxarray(4)(1)){
    maxarray(4)(0) = idx
    maxarray(4)(1) = value
    maxarray = maxarray.sortBy(- _(1))
  }
}
var minarray = Array.ofDim[Double](5,2)
def findmin(idx: Int, value: Double) = {
  if (value < minarray(4)(1)){
    minarray(4)(0) = idx
    minarray(4)(1) = value
    minarray = minarray.sortBy(_(1))
  }
}

  

>     maxarray: Array[Array[Double]] = Array(Array(0.0, 0.0), Array(0.0, 0.0), Array(0.0, 0.0), Array(0.0, 0.0), Array(0.0, 0.0))
>     findmax: (idx: Int, value: Double)Unit
>     minarray: Array[Array[Double]] = Array(Array(0.0, 0.0), Array(0.0, 0.0), Array(0.0, 0.0), Array(0.0, 0.0), Array(0.0, 0.0))
>     findmin: (idx: Int, value: Double)Unit

In [ ]:
lrModel.coefficients.foreachActive((idx, value) => findmax(idx, value))
println(maxarray.deep.foreach(println))
println(vectorizer.vocabulary(maxarray(0)(0).toInt))
println(vectorizer.vocabulary(maxarray(1)(0).toInt))
println(vectorizer.vocabulary(maxarray(2)(0).toInt))
println(vectorizer.vocabulary(maxarray(3)(0).toInt))
println(vectorizer.vocabulary(maxarray(4)(0).toInt))
lrModel.coefficients.foreachActive((idx, value) => findmin(idx, value))
println(minarray.deep.foreach(println))
println(vectorizer.vocabulary(minarray(0)(0).toInt))
println(vectorizer.vocabulary(minarray(1)(0).toInt))
println(vectorizer.vocabulary(minarray(2)(0).toInt))
println(vectorizer.vocabulary(minarray(3)(0).toInt))
println(vectorizer.vocabulary(minarray(4)(0).toInt))

  

>     Array(47.0, 1.9102192494046342)
>     Array(872.0, 1.2777080229192614)
>     Array(377.0, 0.9913337832941463)
>     Array(967.0, 0.9512103966721316)
>     Array(588.0, 0.9075917959122932)
>     ()
>     anonym
>     förlossningen
>     sambon
>     maken
>     himla
>     Array(982.0, -2.000393280148797)
>     Array(866.0, -1.5121327707986945)
>     Array(276.0, -1.4341202867987122)
>     Array(429.0, -1.223628085683817)
>     Array(171.0, -1.093689729083599)
>     ()
>     topic
>     vafan
>     brudar
>     polare
>     jävligt

In [ ]:
val predictions = lrModel.transform(test)
predictions.orderBy(rand()).select("c", "prediction", "probability").show(30, false)


  

>     +---+----------+------------------------------------------+
>     |c  |prediction|probability                               |
>     +---+----------+------------------------------------------+
>     |0  |0.0       |[0.8700974552819184,0.12990254471808155]  |
>     |0  |0.0       |[0.7928008704946266,0.20719912950537347]  |
>     |1  |1.0       |[0.30994317649462944,0.6900568235053705]  |
>     |0  |1.0       |[2.365709712284057E-4,0.9997634290287717] |
>     |1  |1.0       |[0.3864413150551652,0.6135586849448349]   |
>     |1  |1.0       |[0.014691116200145099,0.9853088837998548] |
>     |0  |0.0       |[0.9720829648637792,0.027917035136220764] |
>     |0  |0.0       |[0.7152332415276125,0.2847667584723875]   |
>     |1  |1.0       |[0.151610999721996,0.848389000278004]     |
>     |0  |0.0       |[0.5439103646376973,0.4560896353623028]   |
>     |1  |0.0       |[0.5264273685615647,0.47357263143843537]  |
>     |1  |1.0       |[0.040652161616880136,0.9593478383831199] |
>     |1  |1.0       |[0.4568064824116975,0.5431935175883025]   |
>     |0  |0.0       |[0.8497726939427239,0.1502273060572761]   |
>     |1  |1.0       |[0.16033537277945406,0.839664627220546]   |
>     |0  |0.0       |[0.8411560740277545,0.15884392597224542]  |
>     |1  |1.0       |[0.15523599020127926,0.8447640097987207]  |
>     |0  |0.0       |[0.8179515566406623,0.18204844335933779]  |
>     |0  |0.0       |[0.9999999997092099,2.907900264289424E-10]|
>     |0  |0.0       |[0.5439103646376973,0.4560896353623028]   |
>     |1  |0.0       |[0.7618112961418647,0.23818870385813526]  |
>     |0  |1.0       |[0.3786090173635609,0.6213909826364391]   |
>     |1  |1.0       |[0.012329069380491282,0.9876709306195088] |
>     |0  |0.0       |[0.5439103646376973,0.4560896353623028]   |
>     |0  |0.0       |[0.9266151850746471,0.07338481492535281]  |
>     |1  |1.0       |[1.4285800125598476E-4,0.9998571419987439]|
>     |1  |1.0       |[0.12571197086952507,0.874288029130475]   |
>     |0  |0.0       |[0.5439103646376973,0.4560896353623028]   |
>     |0  |0.0       |[0.9995722995413346,4.277004586653921E-4] |
>     |1  |1.0       |[0.14213744895407485,0.8578625510459252]  |
>     +---+----------+------------------------------------------+
>     only showing top 30 rows
>
>     predictions: org.apache.spark.sql.DataFrame = [c: int, features: vector ... 3 more fields]

In [ ]:
val evaluator = new BinaryClassificationEvaluator().setLabelCol("c")
evaluator.evaluate(predictions)

  

>     evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = BinaryClassificationEvaluator: uid=binEval_d4dc0bb76d07, metricName=areaUnderROC, numBins=1000
>     res96: Double = 0.9244267432611908